In [60]:
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import filedialog
from skimage.segmentation import slic
import matplotlib.pyplot as plt
import numpy as np
from skimage.segmentation import felzenszwalb
from skimage.segmentation import quickshift
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from PIL import ImageFilter
from tkinter import simpledialog

In [64]:
def create_section(title, row, column):
    label = tk.Label(root, text=title, font=('Arial', 12), padx=10, pady=5, cursor='hand2')
    label.grid(row=row, column=column, sticky='w')
    label.bind("<Button-1>", lambda event, title=title: toggle_subsections(title))  # Bind click event

def create_section_with_avatar(image_path, row, column):
    image = Image.open(image_path)
    image = image.resize((32, 32), Image.ANTIALIAS)  # Resize the avatar image
    photo = ImageTk.PhotoImage(image)
    label = tk.Label(avatars_frame, image=photo)
    label.image = photo  # Store a reference to the PhotoImage to prevent garbage collection
    label.grid(row=row, column=column, padx=5, pady=10)

def create_subsection(subsection_title, frame):
    subsection_label = tk.Label(frame, text=subsection_title, font=('Arial', 10), padx=5, pady=3, cursor='hand2')
    subsection_label.grid(sticky='w')
    subsection_label.bind("<Button-1>", lambda event, title=subsection_title: on_subsection_click(title))

def toggle_subsections(title):
    global subsections_frame
    if title == 'Data':
        if subsections_frame is None:
            subsections_frame = tk.Frame(root)
            subsections_frame.grid(row=1, column=0, columnspan=len(sections), padx=5, pady=10, sticky='w')

            subsections = ['Loading Original Image', 'Loading Ground Truths', 'Close']
            for i, subsection_title in enumerate(subsections):
                create_subsection(subsection_title, subsections_frame)
        else:
            subsections_frame.destroy()
            subsections_frame = None
    elif title == 'Pre-processing':
        if subsections_frame is None:
            subsections_frame = tk.Frame(root)
            subsections_frame.grid(row=2, column=0, columnspan=len(sections), padx=5, pady=10, sticky='w')

            subsections = ['Adaptive Filter', 'Resizing', 'Normalization', 'Thresholding', 'Sharpening', 'Denoising', 'Close']
            for i, subsection_title in enumerate(subsections):
                create_subsection(subsection_title, subsections_frame)
        else:
            subsections_frame.destroy()
            subsections_frame = None
    elif title == 'Superpixel':
        if subsections_frame is None:
            subsections_frame = tk.Frame(root)
            subsections_frame.grid(row=3, column=0, columnspan=len(sections), padx=5, pady=10, sticky='w')

            subsections = ['SLIC', 'FH', 'Quick Shift', 'Watershed', 'SEEDS', 'CIS', 'Close']
            for i, subsection_title in enumerate(subsections):
                create_subsection(subsection_title, subsections_frame)
        else:
            subsections_frame.destroy()
            subsections_frame = None
    elif title == 'Machine learning model':
        if subsections_frame is None:
            subsections_frame = tk.Frame(root)
            subsections_frame.grid(row=4, column=0, columnspan=len(sections), padx=5, pady=10, sticky='w')

            subsections = ['U-Net', 'U-Net++', 'Double U-Net', 'Resnet', 'Polar Res-U-Net++', 'DuAT', 'MCGU-Net', 'Attn U-Net + Multi-Input + FTL', 'Close']
            for i, subsection_title in enumerate(subsections):
                create_subsection(subsection_title, subsections_frame)
        else:
            subsections_frame.destroy()
            subsections_frame = None
    elif title == 'Loss function':
        if subsections_frame is None:
            subsections_frame = tk.Frame(root)
            subsections_frame.grid(row=5, column=0, columnspan=len(sections), padx=5, pady=10, sticky='w')

            subsections = ['IOU', 'Dice Coefficient', 'Precision', 'Recall', 'F1 Score', 'Close']
            for i, subsection_title in enumerate(subsections):
                create_subsection(subsection_title, subsections_frame)
        else:
            subsections_frame.destroy()
            subsections_frame = None
    elif title == 'Graph':
        if subsections_frame is None:
            subsections_frame = tk.Frame(root)
            subsections_frame.grid(row=6, column=0, columnspan=len(sections), padx=5, pady=10, sticky='w')

            subsections = ['Plot Histogram', 'Save Graph', 'Close']
            for i, subsection_title in enumerate(subsections):
                create_subsection(subsection_title, subsections_frame)
        else:
            subsections_frame.destroy()
            subsections_frame = None
    elif title == 'Close':
        if subsections_frame is not None:
            subsections_frame.destroy()
            subsections_frame = None

def on_subsection_click(subsection_title):
    global selected_image_path
    if subsection_title == 'Loading Original Image':
        file_path = filedialog.askopenfilename(filetypes=[("PNG Files", "*.png"), ("JPEG Files", "*.jpg;*.jpeg")])
        # Handle the selected file path (e.g., you can display the selected image or process it further)
        if file_path:
            selected_image_path = file_path
            print(f"Selected original image file: {file_path}")
            
            # Display the selected image on the screen
            display_selected_image(selected_image_path)
    elif subsection_title == 'Plot Histogram':
        if selected_image_path:
            plot_histogram(selected_image_path)
        else:
            print("Please select an image first in the 'Loading Original Image' subsection.")
    elif subsection_title == 'Save Graph':
         save_histogram_graph()        

    elif subsection_title == 'Loading Ground Truths':
        file_path = filedialog.askopenfilename(filetypes=[("PNG Files", "*.png"), ("JPEG Files", "*.jpg;*.jpeg")])
        # Handle the selected file path (e.g., you can display the selected image or process it further)
        if file_path:
            print(f"Selected ground truth image file: {file_path}")
    elif subsection_title == 'Adaptive Filter':
        if selected_image_path:
        # Apply adaptive filter to the selected image
           filtered_image = apply_adaptive_filter(selected_image_path)

        # Display the filtered image on the screen
           filtered_image.show()
        else:
           print("Please select an image first in the 'Loading Original Image' subsection.")
# Inside the on_subsection_click function
    elif subsection_title == 'Resizing':
         if selected_image_path:
         # Ask the user for the target size using a dialog box
            target_size_input = simpledialog.askstring("Target Size", "Enter target size (width x height):")
            if target_size_input:
                try: 
                    width, height = map(int, target_size_input.split('x'))
                    target_size = (width, height)
                
                    # Apply resizing to the selected image
                    resized_image = apply_resizing(selected_image_path, target_size)
                
                    # Display the resized image on the screen
                    resized_image.show()
                except ValueError:
                    print("Invalid target size format. Please use 'width x height' format (e.g., 400 x 300).")
            else:
                print("Target size input was canceled.")
         else:
             print("Please select an image first in the 'Loading Original Image' subsection.")
             
    elif subsection_title == 'Normalization':
         if selected_image_path:
        # Ask the user for the normalization range using a dialog box
            min_value = simpledialog.askfloat("Normalization", "Enter minimum value for normalization:", minvalue=0.0)
            max_value = simpledialog.askfloat("Normalization", "Enter maximum value for normalization:", minvalue=0.0)
            if min_value is not None and max_value is not None:
            # Apply normalization to the selected image
               normalized_image = apply_normalization(selected_image_path, min_value, max_value)
            
            # Display the normalized image on the screen
               normalized_image.show()
            else:
               print("Normalization range input was canceled.")
         else:
              print("Please select an image first in the 'Loading Original Image' subsection.")
    elif subsection_title == 'Thresholding':
         if selected_image_path:
              # Ask the user for the threshold value using a dialog box
            threshold_value = simpledialog.askinteger("Thresholding", "Enter threshold value (0-255):", minvalue=0, maxvalue=255)
            if threshold_value is not None:
                   # Apply thresholding to the selected image
                   thresholded_image = apply_thresholding(selected_image_path, threshold_value)
             
                      # Display the thresholded image on the screen
                   thresholded_image.show()
               
            else:
                 print("Threshold value input was canceled.")
             
         else:
            print("Please select an image first in the 'Loading Original Image' subsection.")
        
    elif subsection_title == 'Sharpening':
         if selected_image_path:
            # Apply sharpening to the selected image
            sharpened_image = apply_sharpening(selected_image_path)
        
            # Display the sharpened image on the screen
            sharpened_image.show()
         else:
              print("Please select an image first in the 'Loading Original Image' subsection.")
             
    elif subsection_title == 'Denoising':
         if selected_image_path:
            # Apply denoising to the selected image
             denoised_image = apply_denoising(selected_image_path)
        
           # Display the denoised image on the screen
             denoised_image.show()
         else:
             print("Please select an image first in the 'Loading Original Image' subsection.")
    elif subsection_title == 'SLIC':
        if selected_image_path:
            # Apply SLIC superpixel segmentation to the selected image
            perform_slic_segmentation(selected_image_path)
        else:
            print("Please select an image first in the 'Loading Original Image' subsection.")
    elif subsection_title == 'FH':
         if selected_image_path:
        # Apply Felzenszwalb superpixel segmentation to the selected image
            perform_fh_segmentation(selected_image_path)
         else:
              print("Please select an image first in the 'Loading Original Image' subsection.")

    elif subsection_title == 'Quick Shift':
        if selected_image_path:
           perform_quick_shift_segmentation(selected_image_path)
        else:
             print("Please select an image first in the 'Loading Original Image' subsection.")
         
    elif subsection_title == 'Watershed':
        # Add the functionality for Watershed here
        pass
    elif subsection_title == 'SEEDS':
        # Add the functionality for SEEDS here
        pass
    elif subsection_title == 'CIS':
        # Add the functionality for CIS here
        pass
    elif subsection_title == 'U-Net':
        # Add the functionality for U-Net here
        pass
    elif subsection_title == 'U-Net++':
        # Add the functionality for U-Net++ here
        pass
    elif subsection_title == 'Double U-Net':
        # Add the functionality for Double U-Net here
        pass
    elif subsection_title == 'Resnet':
        # Add the functionality for Resnet here
        pass
    elif subsection_title == 'Polar Res-U-Net++':
        # Add the functionality for Polar Res-U-Net++ here
        pass
    elif subsection_title == 'DuAT':
        # Add the functionality for DuAT here
        pass
    elif subsection_title == 'MCGU-Net':
        # Add the functionality for MCGU-Net here
        pass
    elif subsection_title == 'Attn U-Net + Multi-Input + FTL':
        # Add the functionality for Attn U-Net + Multi-Input + FTL here
        pass
    elif subsection_title == 'IOU':
        # Add the functionality for IOU here
        pass
    elif subsection_title == 'Dice Coefficient':
        # Add the functionality for Dice Coefficient here
        pass
    elif subsection_title == 'Precision':
        # Add the functionality for Precision here
        pass
    elif subsection_title == 'Recall':
        # Add the functionality for Recall here
        pass
    elif subsection_title == 'F1 Score':
        # Add the functionality for F1 Score here
        pass

def perform_slic_segmentation(image_path):
    
    image = Image.open(image_path)

    # Convert the image to numpy array for segmentation
    image_np = np.array(image)

    # Apply SLIC superpixel segmentation
    segments = slic(image_np, n_segments=100, compactness=10, sigma=1)

    # Create a copy of the original image to create grayscale result
    result_image = np.copy(image_np)

    # Assign different grayscale values to each superpixel region
    for segment_id in np.unique(segments):
        grayscale_value = np.random.randint(0, 255)  # Generate random grayscale value for each segment
        result_image[segments == segment_id] = grayscale_value

    # Convert the result back to PIL Image
    result_image = Image.fromarray(result_image)

    # Show the result image with superpixel regions as grayscale
    result_image.show()
    
def perform_fh_segmentation(image_path):
    image = Image.open(image_path)

    # Convert the image to numpy array for segmentation
    image_np = np.array(image)

    # Apply Felzenszwalb superpixel segmentation
    segments = felzenszwalb(image_np, scale=100, sigma=0.5, min_size=50)

    # Create a copy of the original image to create grayscale result
    result_image = np.copy(image_np)

    # Assign different grayscale values to each superpixel region
    for segment_id in np.unique(segments):
        grayscale_value = np.random.randint(0, 255)  # Generate random grayscale value for each segment
        result_image[segments == segment_id] = grayscale_value

    # Convert the result back to PIL Image
    result_image = Image.fromarray(result_image)

    # Show the result image with superpixel regions as grayscale
    result_image.show()
    
def perform_quick_shift_segmentation(image_path):
    image = Image.open(image_path)

    # Convert the image to numpy array for segmentation
    image_np = np.array(image)

    # Apply Quick Shift superpixel segmentation
    segments = quickshift(image_np, kernel_size=3, max_dist=6, ratio=0.5)

    # Create a copy of the original image to create grayscale result
    result_image = np.copy(image_np)

    # Assign different grayscale values to each superpixel region
    for segment_id in np.unique(segments):
        grayscale_value = np.random.randint(0, 255)  # Generate random grayscale value for each segment
        result_image[segments == segment_id] = grayscale_value

    # Convert the result back to PIL Image
    result_image = Image.fromarray(result_image)

    # Show the result image with superpixel regions as grayscale
    result_image.show()
    
def plot_histogram(image_path):
    image = Image.open(image_path)
    image_gray = image.convert("L")  # Convert to grayscale

    # Calculate the histogram
    histogram, bins = np.histogram(image_gray, bins=256, range=(0, 256))

    # Create a new Tkinter window
    histogram_window = tk.Toplevel(root)
    histogram_window.title('Histogram Graph')

    # Create a Matplotlib figure and plot the histogram as a bar graph
    figure, ax = plt.subplots()
    ax.bar(bins[:-1], histogram, width=1, align='center')
    ax.set_title('Histogram')
    ax.set_xlabel('Pixel Value')
    ax.set_ylabel('Frequency')

    # Create a FigureCanvasTkAgg widget to display the Matplotlib figure
    canvas = FigureCanvasTkAgg(figure, master=histogram_window)
    canvas.draw()
    canvas.get_tk_widget().pack()

    # Add a button to close the window
    close_button = tk.Button(histogram_window, text='Close', command=histogram_window.destroy)
    close_button.pack()

    # Run the Tkinter event loop for the histogram window
    histogram_window.mainloop()
    
def save_histogram_graph():
    global selected_image_path
    if selected_image_path:
        image = Image.open(selected_image_path)
        image_gray = image.convert("L")  # Convert to grayscale

        # Calculate the histogram
        histogram, bins = np.histogram(image_gray, bins=256, range=(0, 256))

        # Plot the histogram as a bar graph
        plt.figure()
        plt.bar(bins[:-1], histogram, width=1, align='center')
        plt.title('Histogram')
        plt.xlabel('Pixel Value')
        plt.ylabel('Frequency')

        # Ask the user where to save the graph
        save_path = filedialog.asksaveasfilename(defaultextension='.png', filetypes=[("PNG Files", "*.png")])
        if save_path:
            plt.savefig(save_path)
            print(f"Histogram graph saved as {save_path}")
        else:
            print("Saving was canceled.")
    else:
        print("Please select an image first in the 'Loading Original Image' subsection.")   

def apply_adaptive_filter(image_path):
    image = Image.open(image_path)

    # Apply adaptive filter to the image
    filtered_image = image.filter(ImageFilter.GaussianBlur(radius=2))

    return filtered_image      
def apply_resizing(image_path, target_size):
    image = Image.open(image_path)

    # Resize the image
    resized_image = image.resize(target_size)

    return resized_image

def apply_thresholding(image_path, threshold_value):
    image = Image.open(image_path)
    image_gray = image.convert("L")  # Convert to grayscale
    
    # Apply thresholding to the grayscale image
    thresholded_image = image_gray.point(lambda p: 0 if p < threshold_value else 255)
    
    return thresholded_image

def apply_normalization(image_path, min_value, max_value):
    image = Image.open(image_path)
    image_array = np.array(image)

    # Normalize the image array
    normalized_image_array = (image_array - min_value) / (max_value - min_value) * 255

    # Convert the normalized array back to PIL Image
    normalized_image = Image.fromarray(normalized_image_array.astype('uint8'))

    return normalized_image

def apply_sharpening(image_path):
    image = Image.open(image_path)

    # Apply sharpening filter to the image
    sharpened_image = image.filter(ImageFilter.UnsharpMask(radius=2, percent=150))

    return sharpened_image

def apply_denoising(image_path):
    image = Image.open(image_path)

    # Apply denoising filter to the image
    denoised_image = image.filter(ImageFilter.GaussianBlur(radius=1))

    return denoised_image

def display_selected_image(image_path):
    image = Image.open(image_path)
    image.thumbnail((400, 400))  # Resize the image for display
    photo = ImageTk.PhotoImage(image)
    
    # Create a new window to display the image
    display_window = tk.Toplevel(root)
    display_window.title('Selected Image')
    
    # Create a label to display the image
    image_label = tk.Label(display_window, image=photo)
    image_label.image = photo  # Store a reference to the PhotoImage to prevent garbage collection
    image_label.pack()
    
    # Add a button to close the window
    close_button = tk.Button(display_window, text='Close', command=display_window.destroy)
    close_button.pack()
    
# Create the main window
root = tk.Tk()
root.title('Image Segmentation Software Presentation')

# Calculate the screen width and height
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# Set the window size to half the screen size
window_width = screen_width // 2
window_height = screen_height // 2
root.geometry(f'{window_width}x{window_height}+{window_width//4}+{window_height//4}')  # Center the window on the screen

# Sections
sections = ['Data', 'Pre-processing', 'Superpixel', 'Machine learning model', 'Loss function', 'Graph']

# Create sections in the specified order (from left to right) for the first row
for i, section_title in enumerate(sections):
    create_section(section_title, row=0, column=i)

# Create a frame for avatars
avatars_frame = tk.Frame(root)
avatars_frame.grid(row=1, column=0, columnspan=len(sections), padx=5, pady=10)

# Avatar images for the second row
avatar_images = [
    'new_file.png',
    'save_file.png',
    'undo.png',
    'close.png',
    'backward.png',
    'forward.png'
]

# Create sections with avatars in the specified order (from left to right) for the second row
for i, avatar_path in enumerate(avatar_images):
    create_section_with_avatar(avatar_path, row=0, column=i)

# Add a horizontal line below the first row
line_frame = tk.Frame(root, height=2, bg='black')
line_frame.grid(row=1, column=0, columnspan=len(sections))

subsections_frame = None  # Global variable to store the subsections frame
selected_image_path = ""  # Global variable to store the selected image path

# Run the main event loop
root.mainloop()

/tmp/ipykernel_7596/2031773488.py:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize((32, 32), Image.ANTIALIAS)  # Resize the avatar image


Selected original image file: /home/somayeh/PycharmProjects/pythonProject2/ISBI2016_ISIC_Part1_Training_Data/ISIC_0000026.png
Invalid target size format. Please use 'width x height' format (e.g., 400 x 300).
